# Peer Review 1

#### ALL REQUIRED PACKAGES HAVE BEEN DECLARED HERE

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library
!pip install geocoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Pyth

Get the wiki page containing Toronto,Boroughs AND Neighbourhoods


In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
wikitables = soup.find_all('table') 
Toronto = pd.read_html(str(wikitables[0]), index_col=None, header=0)[0]
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
Toronto.shape

(288, 3)

#### We Begin With Cleaning and Preprocessing of The Data

In [4]:
Toronto['Borough'].replace('', np.nan, inplace=True)
Toronto.dropna(subset=['Borough'], inplace=True)
Toronto = Toronto[Toronto['Borough'] != 'Not assigned']

for i, _ in Toronto.iterrows():
    if Toronto.loc[i]['Neighbourhood'] == 'Not assigned': Toronto.loc[i]['Neighborhood'] = Toronto.loc[i]['Borough']

#### We create dictionary of all postal codes


In [5]:
TPS = Toronto['Postcode'].unique()

len(TPS)

103

In [6]:
!wget -q --no-check-certificate -O 'latitude.pickle' 'https://docs.google.com/uc?export=download&id=1PdEOkPErrpBtDgSlDwczIv_KLlpY-YcO'
!wget -q --no-check-certificate -O 'longitude.pickle' 'https://docs.google.com/uc?export=download&id=1XujA04dCARQnlxu-X2ItOVcYQz0MMQh9'
import pickle

with open('latitude.pickle', 'rb') as latt:
    latitude = pickle.load(latt)
with open('longitude.pickle', 'rb') as lonn:
    longitude = pickle.load(lonn)

In [7]:
TPS = Toronto['Postcode'].unique()

len(TPS)

103

#### for offline availablilty 

In [8]:
import geocoder
import time

for postcode in TPS:
    # When offline cache is available use it to avoid Geocoder Google API throttling
    if postcode in latitude.keys(): 
        continue
    while True:
        g = geocoder.google('{},Toronto, Ontario'.format(postcode))
        latlong_coords = g.latlng
        if latlong_coords == None:
            time.sleep(5)
            continue
        break
    
    if lat_long_coords != None:
        latitude[postcode] = latlong_coords[0]
        longitude[postcode] = latlong_coords[1]
print('Successfully populated geo locs')

Successfully populated geo locs


In [9]:
lat = []
lon = []
for i, _ in Toronto.iterrows():
    lat.append(latitude[Toronto.loc[i]['Postcode']])
    lon.append(longitude[Toronto.loc[i]['Postcode']])
    
Toronto = Toronto.assign(Latitude = lat, Longitude=lon)
Toronto.head()    

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [10]:
Toronto = Toronto.reset_index(drop=True)
Toronto.to_csv('Toronto.csv')

In [11]:
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
T_lat = location.latitude
T_lon = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(T_lat, T_lon))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


#### Now, We Create a map of Toronto along with its neighbourhoods¶


In [13]:
T_map = folium.Map(location=[T_lat, T_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(T_map)  
    
T_map

### foursquare creds

In [14]:
CLIENT_ID = '3BTXCSLDML4CUWEZODUVR3HJHYKTXJEJHJKGDZYDU4ADBWHA'
CLIENT_SECRET = '4EF2EWAYCVQ24WSN4FF05FQAQRAUB3E00MDW2CX2PXUOWID4'
VERSION = '20180323' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3BTXCSLDML4CUWEZODUVR3HJHYKTXJEJHJKGDZYDU4ADBWHA
CLIENT_SECRET:4EF2EWAYCVQ24WSN4FF05FQAQRAUB3E00MDW2CX2PXUOWID4


### we now try and explore 

In [15]:
HighPark = Toronto[Toronto['Borough'] == 'West Toronto']
HighPark.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
1,M6H,West Toronto,Dufferin,43.669005,-79.442259
2,M6J,West Toronto,Little Portugal,43.647927,-79.419750
3,M6J,West Toronto,Trinity,43.647927,-79.419750
4,M6K,West Toronto,Brockton,43.636847,-79.428191
5,M6K,West Toronto,Exhibition Place,43.636847,-79.428191
6,M6K,West Toronto,Parkdale Village,43.636847,-79.428191
7,M6P,West Toronto,High Park,43.661608,-79.464763
8,M6P,West Toronto,The Junction South,43.661608,-79.464763
9,M6R,West Toronto,Parkdale,43.648960,-79.456325


In [16]:
HighPark = Toronto[Toronto['Borough'] == 'West Toronto']
HighPark.reset_index(drop=True)
HighPark_lat = HighPark['Latitude'].values[0] # neighborhood latitude value
print(type(HighPark_lat))
HighPark_lon = HighPark['Longitude'].values[0] # neighborhood longitude value

HighPark_name = HighPark['Neighbourhood'].values[0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(HighPark_name, 
                                                               HighPark_lat, 
                                                               HighPark_lon))

<class 'numpy.float64'>
Latitude and longitude values of Dovercourt Village are 43.6690051, -79.4422593.


In [17]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    HighPark_lat, 
    HighPark_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=3BTXCSLDML4CUWEZODUVR3HJHYKTXJEJHJKGDZYDU4ADBWHA&client_secret=4EF2EWAYCVQ24WSN4FF05FQAQRAUB3E00MDW2CX2PXUOWID4&v=20180323&ll=43.6690051,-79.4422593&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
    

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Happy Bakery & Pastries,Bakery,43.667050,-79.441791
3,FreshCo,Supermarket,43.667918,-79.440754
4,Planet Fitness Toronto Galleria,Gym / Fitness Center,43.667588,-79.442574


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

HP_venues = getNearbyVenues(names=HighPark['Neighbourhood'],
                                   latitudes=HighPark['Latitude'],
                                   longitudes=HighPark['Longitude']
                                  )

Dovercourt Village
Dufferin
Little Portugal
Trinity
Brockton
Exhibition Place
Parkdale Village
High Park
The Junction South
Parkdale
Roncesvalles
Runnymede
Swansea


In [22]:
HP_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dovercourt Village,43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,Dovercourt Village,43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
3,Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Supermarket
4,Dovercourt Village,43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center


In [23]:
HP_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton,21,21,21,21,21,21
Dovercourt Village,20,20,20,20,20,20
Dufferin,20,20,20,20,20,20
Exhibition Place,21,21,21,21,21,21
High Park,24,24,24,24,24,24
Little Portugal,64,64,64,64,64,64
Parkdale,15,15,15,15,15,15
Parkdale Village,21,21,21,21,21,21
Roncesvalles,15,15,15,15,15,15


In [24]:
# type your answer here
HighPark_venues = getNearbyVenues(names=HighPark['Neighbourhood'],
                                   latitudes=HighPark['Latitude'],
                                   longitudes=HighPark['Longitude']
                                  )

Dovercourt Village
Dufferin
Little Portugal
Trinity
Brockton
Exhibition Place
Parkdale Village
High Park
The Junction South
Parkdale
Roncesvalles
Runnymede
Swansea


In [25]:
print(HighPark_venues.shape)
HighPark_venues.head()

(385, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dovercourt Village,43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,Dovercourt Village,43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
3,Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Supermarket
4,Dovercourt Village,43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center


In [26]:
# one hot encoding is taken care of here
HPark_onehot = pd.get_dummies(HighPark_venues[['Venue Category']], prefix="", prefix_sep="")

HPark_onehot['Neighborhood'] = HighPark_venues['Neighborhood'] 

fixed_columns = [HPark_onehot.columns[-1]] + list(HPark_onehot.columns[:-1])
HPark_onehot = HPark_onehot[fixed_columns]

HPark_onehot.head()

,Neighborhood,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Dovercourt Village,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dovercourt Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
HP_grouped = HPark_onehot.groupby('Neighborhood').mean().reset_index()
HP_grouped

,Neighborhood,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.047619,0.000000,0.00000,0.095238,0.000000,0.000000,0.047619,0.095238,0.000000,0.047619,0.047619,0.00000,0.095238,0.047619,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.047619,0.000000,0.047619,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
1,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.05,0.050000,0.000000,0.00000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.00000,0.000000,0.05,0.000000,0.000000,0.050000,0.00000,0.000000,0.050000,0.000000,0.000000,0.100000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.05,0.000000
2,Dufferin,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.05,0.050000,0.000000,0.00000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.00000,0.000000,0.05,0.000000,0.000000,0.050000,0.00000,0.000000,0.050000,0.000000,0.000000,0.100000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.05,0.000000
3,Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.047619,0.000000,0.00000,0.095238,0.000000,0.000000,0.047619,0.095238,0.000000,0.047619,0.047619,0.00000,0.095238,0.047619,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.0

In [28]:
num_top_venues = 5

for hood in HP_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = HP_grouped[HP_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3          Office  0.05
4         Stadium  0.05


----Dovercourt Village----
            venue  freq
0        Pharmacy  0.10
1  Discount Store  0.10
2          Bakery  0.10
3     Supermarket  0.10
4            Café  0.05


----Dufferin----
            venue  freq
0        Pharmacy  0.10
1  Discount Store  0.10
2          Bakery  0.10
3     Supermarket  0.10
4            Café  0.05


----Exhibition Place----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3          Office  0.05
4         Stadium  0.05


----High Park----
                 venue  freq
0                 Café  0.08
1   Mexican Restaurant  0.08
2                  Bar  0.08
3         Antique Shop  0.04
4  Fried Chicken Joint  0.04


----Little Portugal----
                     venue  freq
0                      Bar  0.14
1         Asian Restaurant  0.05
2            

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = HP_grouped['Neighborhood']

for ind in np.arange(HP_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(HP_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brockton,Breakfast Spot,Coffee Shop,Café,Pet Store,Office,Performing Arts Venue,Falafel Restaurant,Convenience Store,Climbing Gym,Caribbean Restaurant
1,Dovercourt Village,Supermarket,Discount Store,Bakery,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Café,Pool,Music Venue,Brewery
2,Dufferin,Supermarket,Discount Store,Bakery,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Café,Pool,Music Venue,Brewery
3,Exhibition Place,Breakfast Spot,Coffee Shop,Café,Pet Store,Office,Performing Arts Venue,Falafel Restaurant,Convenience Store,Climbing Gym,Caribbean Restaurant
4,High Park,Mexican Restaurant,Café,Bar,Antique Shop,Italian Restaurant,Park,Fast Food Restaurant,Cajun / Creole Restaurant,Flea Market,Sandwich Place
5,Little Portugal,Bar,Coffee Shop,Asian Restaurant,Men's Store,Boutique,Restaurant,Pizza Place,Cocktail Bar,Café,Bakery
6,Parkdale,Gift Shop,Breakfast Spot,Cuban Restaurant,Bookstore,Dog Run,Eastern European Restaurant,Movie Theater,Coffee Shop,Restaurant,Burger Joint
7,Parkdale Village,Breakfast Spot,Coffee Shop,Café,Pet Store,Office,Performing Arts Venue,Falafel Restaurant,Convenience Store,Climbing Gym,Caribbean Restaurant
8,Roncesvalles,Gift Shop,Breakfast Spot,Cuban Restaurant,Bookstore,Dog Run,Eastern European Restaurant,Movie Theater,Coffee Shop,Restaurant,Burger Joint
9,Runnymede,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Gastropub,Sandwich Place,Latin American Restaurant,Electronics Store,Falafel Restaurant


In [30]:
clusters = 5
grouped_clustering = HP_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(grouped_clustering)
kmeans.labels_[0:5]

array([1, 3, 3, 1, 4], dtype=int32)

In [31]:
HP_merged = HighPark
HP_merged['Cluster Labels'] = kmeans.labels_
HP_merged = HP_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

HP_merged.head() 

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,1,Supermarket,Discount Store,Bakery,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Café,Pool,Music Venue,Brewery
53,M6H,West Toronto,Dufferin,43.669005,-79.442259,3,Supermarket,Discount Store,Bakery,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,Café,Pool,Music Venue,Brewery
64,M6J,West Toronto,Little Portugal,43.647927,-79.419750,3,Bar,Coffee Shop,Asian Restaurant,Men's Store,Boutique,Restaurant,Pizza Place,Cocktail Bar,Café,Bakery
65,M6J,West Toronto,Trinity,43.647927,-79.419750,1,Bar,Coffee Shop,Asian Restaurant,Men's Store,Boutique,Restaurant,Pizza Place,Cocktail Bar,Café,Bakery
76,M6K,West Toronto,Brockton,43.636847,-79.428191,4,Breakfast Spot,Coffee Shop,Café,Pet Store,Office,Performing Arts Venue,Falafel Restaurant,Convenience Store,Climbing Gym,Caribbean Restaurant


In [32]:
# create map
import sys
sys.setrecursionlimit(1500)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []


for lat, lon, po, cluster in zip(HP_merged['Latitude'], HP_merged['Longitude'], HP_merged['Neighbourhood'], HP_merged['Cluster Labels']):
    label = folium.Popup(str(po) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

RecursionError: maximum recursion depth exceeded in comparison